# File conversions

In [22]:
import ffmpeg

In [72]:
def preprocess_video(input_file, output_file):
    try:
        (
            ffmpeg
            .input(input_file)
            .output(output_file,
                    vcodec='libx264',
                    crf=23,
                    vprofile='baseline',
                    level='3.0',
                    pix_fmt='yuv420p',
                    acodec='aac',
                    ac=2,
                    audio_bitrate='128k',
                    movflags='faststart',
                    )
            .overwrite_output()
            .run()
        )
        print(f"Conversion successful: {output_file}")
    except ffmpeg.Error as e:
        print(f"Error during conversion: {e.stderr.decode()}")

def preprocess_audio(input_file, output_file):
    try:
        (
            ffmpeg
            .input(input_file)
            .output(output_file,
                    # vn=True,  # Disable video stream
                    acodec='pcm_s16le',
                    ar=44100,
                    )
            .overwrite_output()
            .run()
        )
        print(f"Audio extraction successful: {output_file}")
    except ffmpeg.Error as e:
        print(f"Error during audio extraction: ", e)

# Example usage
# preprocess_video('../data/HY_2024_film_01.mp4', '../data_processed/output.mp4')

# Pipelines

In [79]:
import speech_recognition as sr

In [80]:
# list all absolute filenames in ../data dir
import os
from pathlib import Path

PATH =  Path('../data')
TEST_FILENAMES = sorted([Path(PATH / p).absolute() for p in os.listdir(PATH)])
TEST_FILENAMES

[PosixPath('/home/szymon/repos/hy_ai/notebooks/../data/HY_2024_film_01.mp4'),
 PosixPath('/home/szymon/repos/hy_ai/notebooks/../data/HY_2024_film_02.mp4'),
 PosixPath('/home/szymon/repos/hy_ai/notebooks/../data/HY_2024_film_03.mp4'),
 PosixPath('/home/szymon/repos/hy_ai/notebooks/../data/HY_2024_film_04.mp4'),
 PosixPath('/home/szymon/repos/hy_ai/notebooks/../data/HY_2024_film_05.mp4'),
 PosixPath('/home/szymon/repos/hy_ai/notebooks/../data/HY_2024_film_06.mp4'),
 PosixPath('/home/szymon/repos/hy_ai/notebooks/../data/HY_2024_film_07.mp4'),
 PosixPath('/home/szymon/repos/hy_ai/notebooks/../data/HY_2024_film_08.mp4'),
 PosixPath('/home/szymon/repos/hy_ai/notebooks/../data/HY_2024_film_09.mp4'),
 PosixPath('/home/szymon/repos/hy_ai/notebooks/../data/HY_2024_film_10.mp4'),
 PosixPath('/home/szymon/repos/hy_ai/notebooks/../data/HY_2024_film_11.mp4'),
 PosixPath('/home/szymon/repos/hy_ai/notebooks/../data/HY_2024_film_12.mp4'),
 PosixPath('/home/szymon/repos/hy_ai/notebooks/../data/HY_2024_f

In [105]:
AUDIO_FILE = str(TEST_FILENAMES[1])

# 1. Preprocess audio
preprocess_audio(AUDIO_FILE, 
                 '../data_processed/output.wav')

Audio extraction successful: ../data_processed/output.wav


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [106]:
# 2. Feed recognizer
r = sr.Recognizer()
with sr.AudioFile("../data_processed/output.wav") as source:
    audio = r.record(source)  # read the entire audio file

In [107]:
# 3. Recodnize speech
prompt = ""

try:
    output = r.recognize_whisper(
        model="small",
        audio_data=audio, 
        language="polish",
        word_timestamps=False,
        show_dict=False,
        prompt=prompt,
        )
except sr.UnknownValueError:
    print("could not understand audio")
except sr.RequestError as e:
    print("error; {0}".format(e))

/home/szymon/repos/hy_ai/hy_ai/lib/python3.10/site-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=d

In [101]:
output

' W pierwszej połowie lipca przeprowadziliśmy ogólnopolską akcję z możonej kontroli przesyłek pocztowych oraz kurierskich. Funkcjonariusze przeprowadzili kontrolę w 18 punktach w całej Polsce. Wrześniowa zmiana warunków oprezentowania obligacji oszczędnościowych wynika z potrzeby ich do stosowania do bieżących realiów rynkowych.'